# FORECAST: Новости → Тикеры → Влияние на свечи (Kaggle)

Перед запуском:
- В Settings → Accelerator включите GPU (опционально, можно и CPU).
- Загрузите ваш проект как Kaggle Dataset (весь каталог с .py скриптами и CSV).

Ноутбук автоматически найдёт каталог в `/kaggle/input/` и скопирует файлы в `/kaggle/working`. Дальше:
1) Установка зависимостей
2) Копирование проекта
3) Авторазметка `tickers` для новостей
4) Обучение нейросети (BiGRU+attention)
5) Инференс: агрегаты влияния новостей на свечи


In [ ]:
pip install -q pandas==2.2.2 numpy==1.26.4 pyarrow==16.1.0 scikit-learn==1.5.1 razdel==0.5.0 pymorphy3==2.0.2 pymorphy3-dicts-ru==2.4.417150.4580142 rapidfuzz==3.9.6 emoji==2.12.1 text-unidecode==1.3 torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121


In [ ]:
import os, shutil, glob
from pathlib import Path

INPUT_ROOT = Path('/kaggle/input')
WORK = Path('/kaggle/working')

# Найдём ваш датасет по имени каталога с проектом (ищем news_analize)
src = None
for p in INPUT_ROOT.glob('**/*'):
    if p.is_dir() and p.name.lower().startswith('news_analize'):
        src = p
        break
if src is None:
    # fallback: если датасет назван иначе, берём первый каталог
    for p in INPUT_ROOT.iterdir():
        if p.is_dir():
            src = p
            break
print('SRC:', src)

# Копируем файлы проекта в рабочую директорию
for f in src.rglob('*'):
    rel = f.relative_to(src)
    dest = WORK / rel
    if f.is_dir():
        dest.mkdir(parents=True, exist_ok=True)
    else:
        dest.parent.mkdir(parents=True, exist_ok=True)
        shutil.copy2(f, dest)
print('Files copied to', WORK)



In [ ]:
# Авторазметка тикеров
!python auto_label_tickers.py --news test_news.csv --out task_1_news_autolabeled.csv

# Быстрая проверка наличия столбца tickers
import pandas as pd
x = pd.read_csv('task_1_news_autolabeled.csv')
print(x.columns.tolist()[:10])
print(x[['publish_date','title','tickers']].head())


In [ ]:
# Обучение модели новости→тикеры
!python train_news_ticker.py --news task_1_news_autolabeled.csv --artifacts artifacts/ --epochs 3 --batch_size 64 --max_len 256

# Проверим артефакты
import os
print('Artifacts:', os.listdir('artifacts'))


In [ ]:
# Инференс влияния на свечи
# При необходимости замените имена CSV на ваши (public_test_candles.csv / task_1_candles.csv)
!python infer_news_to_candles.py \
  --news task_1_news_autolabeled.csv \
  --candles public_test_candles.csv \
  --artifacts artifacts/ \
  --out nn_features.parquet

import pandas as pd
print(pd.read_parquet('nn_features.parquet').head())
